In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
from train import train

In [ ]:
DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/patient_rep/combat_processed.h5ad'

In [ ]:
adata = sc.read_h5ad(DATA_PATH)

In [ ]:
train(adata)